In [1]:
# Script to generate variations on David's 2-BTP model to run methane flame speeds to compare with NIST
# Just provide it with a list of reactions indices from the rmgpy.chemkin loaded RMG_reaction_list

In [63]:
rmg_reactions_to_mix = [192, 586, 240, 663, 175, 176, 139, 181, 52, 110, 170, 211, 258, 669, 520, 133, 570, 80, 532, 582, 121, 171, 202, 277, 20, 257, 14, 222, 4, 11]

In [59]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
import numpy as np
import subprocess

import mix_thermokinetics

import scipy
import copy
import os

In [4]:
# load David's 2-BTP model, minus the halogens
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG_no_halogens/no_halogens.inp'
RMG_dictionary_path = 'models/RMG_no_halogens/no_halogens_dict.txt'
RMG_transport_path = 'models/RMG_no_halogens/no_halogens_tran.dat'
RMG_cti_path = 'models/RMG_no_halogens/no_halogens.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)


# Load NIST model without halogens
NIST_cti_path = 'models/NIST_no_halogens/no_halogens.cti'
NIST_chemkin_path = 'models/NIST_no_halogens/no_halogens.inp'
NIST_dictionary_path = 'models/NIST_no_halogens/no_halogens_dict.txt'
NIST_transport_path = 'models/NIST_no_halogens/no_halogens_tran.dat'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path, transport_path=NIST_transport_path)



For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [5]:
# Species Diff
common_species = []
RMG2NIST_mapping = {}
NIST2RMG_mapping = {}
for i, rmg_sp in enumerate(RMG_species_list):
    for j, nist_sp in enumerate(NIST_species_list):
        if rmg_sp.is_isomorphic(nist_sp):
            RMG2NIST_mapping[i] = j
            NIST2RMG_mapping[j] = i
            common_species.append([rmg_sp, nist_sp])
            break

# Reaction Diff
common_reactions = []
RMG2NIST_rxn_mapping = {}
NIST2RMG_rxn_mapping = {}
for i, rmg_rxn in enumerate(RMG_reaction_list):
    for j, nist_rxn in enumerate(NIST_reaction_list):
        if rmg_rxn.is_isomorphic(nist_rxn):
            RMG2NIST_rxn_mapping[i] = j
            NIST2RMG_rxn_mapping[j] = i
            common_reactions.append([rmg_rxn, nist_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

45 common species
240 common reactions


In [6]:
# Convert the NIST species in the reactions to RMG species, but keep the NIST kinetics
def NIST2RMG(nist_reaction):
    rmg_reaction = copy.deepcopy(nist_reaction)
    reactants = []
    for reactant in nist_reaction.reactants:
        try:
            NIST_species_index = NIST_species_list.index(reactant)
            reactants.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if reactant in RMG_species_list:
                reactants.append(reactant)
        
    rmg_reaction.reactants = reactants
    
    products = []
    for product in nist_reaction.products:
        try:
            NIST_species_index = NIST_species_list.index(product)
            products.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if product in RMG_species_list:
                products.append(product)
    rmg_reaction.products = products
    
    return rmg_reaction


In [7]:
# convert all the NIST reactions to use RMG species definitions
for key in RMG2NIST_rxn_mapping.keys():
    nist_index = RMG2NIST_rxn_mapping[key]
    NIST_reaction_list[nist_index] = NIST2RMG(NIST_reaction_list[nist_index])

In [8]:
# Generate the random numbers to do the mixing
N = len(rmg_reactions_to_mix)
assert N == 30  # you'll run into reproducibility issues if you keep changing this up
# num_flame_speeds = 2 ** 11
num_flame_speeds = 2 ** 5
sampler = scipy.stats.qmc.Sobol(d=N, scramble=True, seed=400)
sample = sampler.random(num_flame_speeds)
sample.shape

(32, 30)

In [9]:
# make a place for the models to live
model_dir = 'mixed_models'
os.makedirs(model_dir, exist_ok=True)


In [10]:
RMG_reaction_list[0].kinetics.A.value

98410000000000.0

In [84]:
# Create a new model with the mixed reactions

# only need to save transport and dictionary once
transport_file = os.path.join(model_dir, f'tran.dat')
dict_file = os.path.join(model_dir, f'species_dictionary.txt')
rmgpy.chemkin.save_transport_file(transport_file, RMG_species_list)
rmgpy.chemkin.save_species_dictionary(dict_file, RMG_species_list)

# for i in range(0, num_flame_speeds):
for i in range(0, 1):
    new_RMG_reactions = copy.deepcopy(RMG_reaction_list)
#     for j in range(0, len(rmg_reactions_to_mix)):
    for j in range(0, 30):
        
        if j == 8 or j==12 or j==13:
            continue
        
        rmg_index = rmg_reactions_to_mix[j]
        
        rmg_reaction = new_RMG_reactions[rmg_index]
        
        
        Ar = RMG_species_list[9]
        He = RMG_species_list[10]
        # skip argon and helium for right now
        if Ar in rmg_reaction.reactants or He in rmg_reaction.reactants or rmg_index == 175 or rmg_index == 176:
            continue
        try:
            nist_reaction = NIST_reaction_list[RMG2NIST_rxn_mapping[rmg_index]]
        except KeyError:
            nist_reaction = copy.deepcopy(rmg_reaction)
            nist_reaction.kinetics.A.value = 0.0
        
        
        if Ar in nist_reaction.reactants or He in nist_reaction.reactants:
            continue

#         print(nist_reaction, rmg_reaction)
        w = sample[i, j]  # each row of sample is an array of weights for one model
        try:

            new_kinetics = mix_thermokinetics.mix_kinetics(rmg_reaction, nist_reaction, w)
            new_RMG_reactions[j].kinetics = new_kinetics
        except AssertionError:
            continue
        print(rmg_index)
    
    # save each model as a cti file
    chemkin_file = os.path.join(model_dir, f'chem_{i:04}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, RMG_species_list, new_RMG_reactions, verbose=True, check_for_duplicates=True)
#     subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_file}', f'--output=model_{i:04}.cti'])
    


192
586
240
663
139
181
520
133
570
532
582
277
257
14


In [55]:
rmg_reactions_to_mix[4]

175

In [56]:
RMG_reaction_list[175]

In [53]:
RMG_species_list[9]

In [32]:
for j in rmg_reactions_to_mix:
    print(RMG_reaction_list[j])

H(5) + C2H3(30) <=> H2(8) + C2H2(24)
C3H3(4828) + propyne(2596) <=> C3H3(4828) + allene(5216)
CH3(20) + C2H4(31) <=> CH4(3) + C2H3(30)
O(6) + butadiene13(26591) <=> OH(7) + CH2CHCHCH(355)
OH(7) + C2H2(24) <=> H2O(9) + C2H(23)
OH(7) + C2H2(24) <=> CO(14) + CH3(20)
H(5) + CH3OH(27) <=> H2(8) + CH3O(28)
H(5) + CH2CO(29) <=> H2(8) + HCCO(22)
OH(7) + CH2(T)(19) <=> H2O(9) + CH(18)
CH3O(28) <=> H(5) + CH2O(21)
C2H2(24) <=> H2CC(25)
CH3CO(35) <=> CO(14) + CH3(20)
HO2(12) + C2H6(32) <=> H2O2(13) + C2H5(33)
propen1yl(195) <=> propen2yl(2598)
H2(8) + C3H3(4828) <=> H(5) + propyne(2596)
HO2(12) + CH4(3) <=> H2O2(13) + CH3(20)
H2(8) + C3H3(4828) <=> H(5) + allene(5216)
CH2O(21) <=> H2(8) + CO(14)
CH4(3) + C3H3(4828) <=> CH3(20) + propyne(2596)
CH4(3) + C3H3(4828) <=> CH3(20) + allene(5216)
CH2OH(34) <=> H(5) + CH2O(21)
H(5) + C2H2(24) <=> C2H3(30)
CH2CHO(36) <=> H(5) + CH2CO(29)
O(6) + C2H4(31) <=> OH(7) + C2H3(30)
H2O2(13) <=> OH(7) + OH(7)
O2(4) + C2H6(32) <=> HO2(12) + C2H5(33)
H(5) + HO2(12) <

In [19]:
new_RMG_reactions[4].kinetics

Arrhenius(A=(14.4145,'m^3/(mol*s)'), n=2.04662, Ea=(62.8391,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(3000,'K'), comment="""Fitted to 1001 data points; dA = *|/ 1, dn = +|- 1.02783e-15, dEa = +|- 8.09094e-15 kJ/mol""")

In [21]:
RMG_reaction_list[4].kinetics

ThirdBody(arrheniusLow=Arrhenius(A=(4.58e+19,'cm^3/(mol*s)'), n=-1.4, Ea=(104.39,'kcal/mol'), T0=(1,'K')), efficiencies={Molecule(smiles="O"): 12.02, Molecule(smiles="[He]"): 0.0, Molecule(smiles="[H][H]"): 2.55, Molecule(smiles="[Ar]"): 0.0, Molecule(smiles="[C-]#[O+]"): 1.95, Molecule(smiles="O=C=O"): 3.83, Molecule(smiles="C=O"): 2.5, Molecule(smiles="CO"): 3.0, Molecule(smiles="CC"): 3.0, Molecule(smiles="C"): 2.0, Molecule(smiles="N#N"): 1.01})

In [ ]:
RMG_reaction_list[5].kinetics.A.get_conversion_factor_from_si_to_cm_mol_s()

In [ ]:
new_RMG_reactions[4].kinetics.A.get_conversion_factor_from_si_to_cm_mol_s()

In [ ]:
# for key in RMG2NIST_rxn_mapping.keys():
#     print(key, RMG_reaction_list[key])

type(RMG_reaction_list[5].kinetics)

In [ ]:
rxn1

In [ ]:
RMG_reaction_list[14].kinetics.A.units

In [ ]:
str(new_RMG_reactions[8])

In [ ]:
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_file}'])

# cantera.ck2cti.convertMech(os.path.join(RMG_NIST_path, 'RMG_NIST.inp'), transport=os.path.join(RMG_NIST_path, 'RMG_NIST_tran.dat'))

In [ ]:
common_species

In [ ]:
# create a model that only has RMG-NIST common reactions
common_path = os.path.join('models', 'common')
os.makedirs(common_path, exist_ok=True)

common_species = []
common_reactions = []
for i in range(0, len(RMG_species_list)):
    if i in RMG2NIST_mapping.keys():
        common_species.append(RMG_species_list[i])

for i in range(0, len(RMG_reaction_list)):
    if i in RMG2NIST_rxn_mapping.keys():
        common_reactions.append(RMG_reaction_list[i])

    
chemkin_file = os.path.join(common_path, 'common.inp')
transport_file = os.path.join(common_path, 'common.dat')
        

rmgpy.chemkin.save_chemkin_file(chemkin_file, common_species, common_reactions, verbose=True, check_for_duplicates=True)
rmgpy.chemkin.save_transport_file(transport_file, common_species)
rmgpy.chemkin.save_species_dictionary(os.path.join(common_path, 'common_dict.txt'), common_species)

subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_file}'])

In [ ]:
dir(cantera.ck2cti)

In [ ]:
len(RMG_reaction_list)

In [87]:
for rxn in common_reactions:
    print(str(rxn))

[LibraryReaction(index=1, reactants=[Species(index=4, label="O2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.53732,-0.0012157,5.31615e-06,-4.8944e-09,1.45844e-12,-1038.59,4.68369], Tmin=(100,'K'), Tmax=(1074.56,'K')), NASAPolynomial(coeffs=[3.15383,0.00167803,-7.69968e-07,1.51274e-10,-1.08781e-14,-1040.82,6.16752], Tmin=(1074.56,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: primaryThermoLibrary"""), molecule=[Molecule(smiles="[O][O]")], transport_data=TransportData(shapeIndex=1, epsilon=(887.154,'J/mol'), sigma=(3.467,'angstrom'), dipoleMoment=(0,'De'), polarizability=(0,'angstrom^3'), rotrelaxcollnum=0.0, comment="""PrimaryTransportLibrary"""), molecular_weight=(31.9988,'amu')), Species(index=5, label="H", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[2.5,-2.38914e-13,3.12709e-16,-1.33367e-19,1.7499e-23,25474.2,-0.444973], Tmin=(100,'K'), Tmax=(4383.16,'K')), NASAPolynomial(coeffs=[2.50003,-3.04997e-08,1.01101e-11,-1.48797e-15,8.20356e-2

[TemplateReaction(index=578, reactants=[Species(index=8, label="H2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.43536,0.00021271,-2.78625e-07,3.40267e-10,-7.76032e-14,-1031.36,-3.90842], Tmin=(100,'K'), Tmax=(1959.08,'K')), NASAPolynomial(coeffs=[2.78817,0.00058764,1.59011e-07,-5.52739e-11,4.34311e-15,-596.144,0.112731], Tmin=(1959.08,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: primaryThermoLibrary"""), molecule=[Molecule(smiles="[H][H]")], transport_data=TransportData(shapeIndex=1, epsilon=(496.374,'J/mol'), sigma=(2.833,'angstrom'), dipoleMoment=(0,'De'), polarizability=(0,'angstrom^3'), rotrelaxcollnum=0.0, comment="""PrimaryTransportLibrary"""), molecular_weight=(2.01594,'amu')), Species(index=4828, label="C3H3", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.09019,0.0173588,-8.30759e-06,-2.47873e-09,2.58719e-12,40755.8,8.11426], Tmin=(100,'K'), Tmax=(949.36,'K')), NASAPolynomial(coeffs=[6.99885,0.0072442,-2.36574e-06,3.98596e-10,-